# Memory
## Setup

First, let's connect to our SQLite database containing music store data.


In [ ]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

Define a context schema to hold customer information (first and last name) that will be available throughout the agent's execution.


In [ ]:
import { z } from "zod";

const contextSchema = z.object({
    first: z.string(),
    last: z.string(),
});
type Context = z.infer<typeof contextSchema>;

Create a tool to execute SQL queries. It supports named parameters (`:first`, `:last`) that get replaced with values from the runtime context.


In [ ]:
import { tool, Runtime } from "langchain";

const executeSQL = tool(async ({ query }, runtime: Runtime<Context>) => {
    const placeholders = query.match(/:(\w+)/g);
    if (!placeholders) {
        return await db.run(query);
    }

    const updatedQuery = placeholders.reduce((acc, placeholder) => {
        const key = placeholder.slice(1) as keyof Context;
        return acc.replace(placeholder, `"${runtime.context[key]}"`);
    }, query);

    return await db.run(updatedQuery);
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results. Named parameters like :first and :last will be filled from runtime context.",
    schema: z.object({
        query: z.string()
    })
});

Add a currency conversion tool for converting USD amounts to other currencies.


In [ ]:
import { tool } from "langchain";

const FALLBACK_RATES = { USD: 1.0, EUR: 0.92, JPY: 150.0, GBP: 0.79 }

const convertCurrency = tool(async ({ amount, toCurrency }) => {
    try {
        const url = `https://api.exchangerate.host/convert?from=USD&to=${toCurrency}&amount=${amount}`
        const response = await fetch(url, {
            headers: {
                "Accept": "application/json"
            }
        })
        const data = await response.json()
        const result = data.result
        if (result) {
            return Math.round(amount * FALLBACK_RATES[to_currency], 2)
        }
    } catch (error) {
        console.error(`[convert_currency] Falling back due to error: ${error}`)
    }
    if (!(toCurrency in FALLBACK_RATES)) {
        throw new Error("Unsupported currency in fallback mode")
    }
    return Math.round(amount * FALLBACK_RATES[toCurrency], 2)
}, {
    name: "convert_currency",
    description: `
        Convert an amount in USD to another currency using live exchange rates.
        Always use this tool if the user requests a currency different 
        from the one stored in the database (USD).
    `,
    schema: z.object({
        amount: z.number(),
        toCurrency: z.string()
    })
});

Define the system prompt that instructs the agent how to interact with the database safely and use named parameters.


In [ ]:
export const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If a query involves a specific customer, always use the named parameters
  :first and :last (instead of hard-coding values).
  Example: SELECT InvoiceId, Total FROM Invoice
           JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
           WHERE Customer.FirstName = :first AND Customer.LastName = :last;
  Assume the customer first and last name are known and will be inserted into the query later.`

Create the agent with our tools and system prompt. Note: **no checkpointer yet**, so the agent won't remember previous conversations.


In [ ]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5",
    tools: [convertCurrency, executeSQL],
    systemPrompt: SYSTEM,
    contextSchema,
    middleware: []
})

## Repeated Queries

Ask about Frank Harris's last invoice. The agent successfully retrieves the information.


In [ ]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What was the total on my last invoice?");
const result = await agent.invoke({
    messages: [question]
}, {
    context: { first: "Frank", last: "Harris" },
    streamMode: "values",
});

result.messages.at(-1).content;

Now ask a follow-up question: "What were the titles?" Without memory, the agent has no idea what we're referring to and asks for clarification.


In [ ]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What were the titles?");
const result = await agent.invoke({
    messages: [question]
}, {
    context: { first: "Frank", last: "Harris" },
    streamMode: "values",
});

result.messages.at(-1).content;

## Add memory

Let's fix this by adding a checkpointer. The `MemorySaver` will store conversation history so the agent can remember context across turns.


In [ ]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";
import { MemorySaver } from "@langchain/langgraph";

const checkpointer = new MemorySaver()
const agent = createAgent({
    model: "openai:gpt-5",
    tools: [convertCurrency, executeSQL],
    systemPrompt: SYSTEM,
    contextSchema,
    checkpointer,
    middleware: []
})

Ask the same question again, but this time with a `thread_id` to track the conversation.


In [ ]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What was the total on my last invoice?");
const result = await agent.invoke({
    messages: [question]
}, {
    context: { first: "Frank", last: "Harris" },
    configurable: { thread_id: "1" },
    streamMode: "values",
});

result.messages.at(-1).content;

Now when we ask "What were the titles?", the agent remembers the previous question about the invoice and returns the track titles! 🎵


In [ ]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What were the titles?");
const result = await agent.invoke({
    messages: [question]
}, {
    context: { first: "Frank", last: "Harris" },
    configurable: { thread_id: "1" },
    streamMode: "values",
});

result.messages.at(-1).content;